# Impementation of Catboost

To solve this problem I descided on using CatBoost Classifier. CatBoost is a decision tree clasifier that accells on categorical features.

In [20]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

## Metadata and parameters

For the model to be able to distinguish between continous, number valriables and categorical values a list of string varibles, as well as some "categorical-looking" variables are passed in to a list. More on this choice in the other notebook.

After importing the dataset all nan values are sett to -1, a value not used in the dataset, here used to make a new category representing NaN values for the model. Afther that the dataset is split into X and Y sets, or variable and target sets.


In [21]:
categoricalVars = [0, 1, 5, 7, 9, 11, 12, 14, 15, 23, 24, 27, 2, 3, 13, 18, 20, 26]
learningRate = 0.01

addHexVars = False
addIntCategorcalVars = False
iterations = 2500
depth = 11
seed = 1
evalSize = 0.15

dataTrain = pd.read_csv("data/challenge1_train.csv")

noNans = dataTrain.fillna(0)

values = noNans.values
X = values[:,2:31]
Y = values[:,1]


## Categorical int vars

This is togglable method if one wants to use some of the number variables as categories. To do this the variables must be converted to int to be accepted by CatBoost

In [22]:
intVars = [6, 8, 22, 28]

if addIntCategorcalVars:
    for col in intVars:
        categoricalVars.append(col)
        for i, item in enumerate(X[:,col]):
            X[:,col][i] = int(item)

## Hex vars

This is a togglable method to convert hexadecimal values to decimal numbers and make them continous variables, not categorical 

In [23]:
hexVars = [2, 3, 13, 18, 20, 26]

if addHexVars:
    for col in hexVars:
        categoricalVars.remove(col)
        for i, item in enumerate(X[:,col]):
            if "E+" in str(item):
                item = int(float(str(item)))
            X[:,col][i] = int(str(item), 16)

## Initiaing the model

Parameters are passed in to the model

In [24]:
model = CatBoostClassifier(
    iterations = iterations,
    depth = depth,
    learning_rate=learningRate,
    eval_metric="AUC",
    early_stopping_rounds=100,
    loss_function='Logloss',
    verbose=100
)

## 'train_test_split' and 'fit'

'train_test_split' splits data into two sets, one for training and fitting the model and one for evaluation. 
For highest score this should be ignored and X and Y should be passed in to 'model.fit' in place of xTrain and yTrain. 
Though there is a risk of overfitting this results in a moddel trained on the most data giving the highest score in this (or in my) case. 
To be able to display a nice graph with estimation of score a eval_set that is not contained within the training set should be passed in to 'model.fit'
'train_test_split' was valuable during the tunig phase as one could get a good estimate of the accuracy of the model without submitting an atempt.

'model.fit' uses 'logloss' to minimize the difference between estimation and target in the trainingset.

In [25]:
xTrain, xTest, yTrain, yTest = train_test_split(X, Y, test_size=evalSize, random_state=seed)
model.fit(X, Y, cat_features=categoricalVars, plot = True)
# model.fit(xTrain, yTrain, cat_features=categoricalVars, eval_set=(xTest, yTest), plot = True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	total: 46.6ms	remaining: 2m 19s
100:	total: 24.5s	remaining: 11m 43s
200:	total: 55.4s	remaining: 12m 50s
300:	total: 1m 29s	remaining: 13m 20s
400:	total: 2m 3s	remaining: 13m 19s
500:	total: 2m 37s	remaining: 13m 5s
600:	total: 3m 12s	remaining: 12m 47s
700:	total: 3m 47s	remaining: 12m 26s
800:	total: 4m 22s	remaining: 12m 1s
900:	total: 5m	remaining: 11m 39s
1000:	total: 5m 36s	remaining: 11m 11s
1100:	total: 6m 12s	remaining: 10m 43s
1200:	total: 6m 49s	remaining: 10m 13s
1300:	total: 7m 26s	remaining: 9m 42s
1400:	total: 8m 3s	remaining: 9m 11s
1500:	total: 8m 40s	remaining: 8m 39s
1600:	total: 9m 18s	remaining: 8m 8s
1700:	total: 9m 57s	remaining: 7m 36s
1800:	total: 10m 36s	remaining: 7m 3s
1900:	total: 11m 18s	remaining: 6m 32s
2000:	total: 12m 1s	remaining: 6m
2100:	total: 12m 42s	remaining: 5m 26s
2200:	total: 13m 23s	remaining: 4m 51s
2300:	total: 14m 4s	remaining: 4m 16s
2400:	total: 14m 45s	remaining: 3m 40s
2500:	total: 15m 27s	remaining: 3m 5s
2600:	total: 16m 9s	rem

## Predicting the test data set

Before we can use the model to make predictions on the test set we need to apply the same opperations to the train set as we have done to the train set.

In [26]:
testData = pd.read_csv("data/challenge1_test.csv").fillna(0)
testData.drop(columns=testData.columns[0], axis=1, inplace=True)
testData = testData.values

if addIntCategorcalVars:
    for col in intVars:
        for i, item in enumerate(testData[:,col]):
            testData[:,col][i] = int(item)

if addHexVars:
    for col in hexVars:
        for i, item in enumerate(testData[:,col]):
            if "E+" in str(item):
                item = int(float(str(item)))
            testData[:,col][i] = int(str(item), 16)


## Predictions

Predictions made by the model is passed out as a probability distribution of the different classes the model are classifying (0, 1).

In [27]:
predictions = model.predict_proba(testData)
predictions

array([[0.76937343, 0.23062657],
       [0.86605208, 0.13394792],
       [0.88376238, 0.11623762],
       ...,
       [0.91861852, 0.08138148],
       [0.88467169, 0.11532831],
       [0.91020712, 0.08979288]])

## Writing to file

Finaly the prediction (probability of the output being a 1) is written to a csv file along with the id of the row the prediction is made on

In [28]:
out = {"id": [], "target": []}
for i, prediction in enumerate(predictions):
    out["id"].append(i+50000)
    out["target"].append(prediction[1])
df = pd.DataFrame(out) 
df.to_csv('Predictions.csv', index=False) 